# Foreign Exchange Intervention Rules for Central Banks: A Ris-Based Framework

This notebook replicates the tables and the charts of the IMF WP on Foreign Exchange Intervention Rules for Central Banks: A Ris-Based Framework

It uses a Python package that I have written, DistGARCH, also available in this Github folder, with the public FX intervention data from the Banco Mexico. DistGARCH is based on the ARCH package of Kevin Sheppard.

You can use the code for non-commercial applications, providing that you cite the  IMF Working Paper\
Lafarguette, R. and Veyrune, R. (2020) *Foreign Exchange Intervention Rules for Central Banks: A Ris-Based Framework*,
IMF Working Paper no 20XX

Author: Romain Lafarguette, September 2020\
If you have any question, please contact me via Github or rlafarguette "at" imf "dot" org

## Preamble

In [44]:
# System paths
import os, sys
sys.path.append(os.path.abspath('modules'))

# Global modules
import importlib                                        # Operating system
import pandas as pd                                     # Dataframes
import numpy as np                                      # Numeric Python
import datetime                                         # Dates
import arch                                             # ARCH/GARCH models

# Functional imports
from datetime import datetime as date                   # Short date function
from dateutil.relativedelta import relativedelta        # Dates manipulation 

# ARCH package functional imports
from arch.univariate import ARX # Drift model
from arch.univariate import (ARCH, GARCH, EGARCH, EWMAVariance, # Vol process
                             FixedVariance, RiskMetrics2006) 
from arch.univariate import (Normal, StudentsT, # Distribution of residuals
                             SkewStudent, GeneralizedError)

# Local modules
import distGARCH; importlib.reload(distGARCH)           # Distributional GARCH
from distGARCH import DistGARCH

# Graphics
%matplotlib inline
import matplotlib.pyplot as plt                         # Graphical package  
import seaborn as sns                                   # Graphical tools

plt.rcParams["figure.figsize"]=25,15

sns.set(style='white', font_scale=4, palette='deep', font='serif',
        rc={'text.usetex' : True}) 

# Ignore a certain type of warnings which occurs in ML estimation
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Figure 1: Conditional VaR and density (diagram)

<div>
<img src="img/VaR_concept.PNG" width="800"/>
</div>


In [45]:
# Read the data for Mexico
df = pd.read_csv(os.path.join('data', 'MXN_data.csv'), parse_dates=['date'], index_col=[0])

# Read the interventions dataframe
dmx = pd.read_csv(os.path.join('data', 'intervention_data.csv'), parse_dates=['date'], index_col=0)

# Merge with the MXN peso database
dfxi = pd.merge(df, dmx[['Sell amount', 'type']],
                left_index=True, right_index=True, how='outer')

# Create FX intervention variables
dfxi['FX intervention in USD'] = dfxi['Sell amount'].fillna(0)
dfxi['fx_intervention_minprice'] = dfxi.loc[dfxi['type']=='min price', 'Sell amount'].fillna(0)
dfxi['fx_intervention_nominprice'] = dfxi.loc[dfxi['type']=='no min price', 'Sell amount'].fillna(0)

dfxi['FX intervention dummy'] = 0
dfxi.loc[dfxi['FX intervention in USD']>0, 'FX intervention dummy'] = 1

dfxi['FX intervention in USD lag'] = dfxi['FX intervention in USD'].shift(1) 
dfxi['FX intervention dummy lag'] = dfxi['FX intervention dummy'].shift(1)


## Fit the GARCH model for different specifications

In [46]:
# Prepare the list of variables
constant = []

microstructure = ['Bid-ask spread abs value',
                  'Forward points first difference']

cip = microstructure + ['Interbank rate vs Libor']

fxi = cip + ['FX intervention in USD lag']

baseline = fxi + ['VIX first diff', 'EURUSD log returns']

robustness = cip + ['FX intervention dummy lag'] + ['VIX first diff', 'EURUSD log returns']

# List of models
models_l = [constant, microstructure, cip, fxi, baseline, robustness]
labels_l = ['Constant', 'Microstructure', 'CIP', 'FXI', 'Baseline', 'Robustness']

specification_tables_l = list()
specification_tables_short_l = list()
for label, model in zip(labels_l, models_l):
    dgm = DistGARCH(depvar_str='FX log returns',
                    data=dfxi,
                    level_str='FX level', 
                    exog_l=model, 
                    lags_l=[1], 
                    vol_model=EGARCH(1,1,1),
                    dist_family=SkewStudent())

    # Fit the model
    dgfit = dgm.fit()

    # Generate the tables
    var_d = {'FX l...rns[1]':'Lag FX log returns'}
    sumtable = dgfit.summary_table(model_name=label, var_d=var_d, print_pval=True)
    sumtable_short = dgfit.summary_table(model_name=label, var_d=var_d, print_pval=False)
    specification_tables_l.append(sumtable)
    specification_tables_short_l.append(sumtable_short)
    
# Merge all the summary tables (need to reorder some rows)
dsum = pd.concat(specification_tables_l, axis=1)
dsum_short = pd.concat(specification_tables_short_l, axis=1)

                                AR - EGARCH Model Results                                
Dep. Variable:                    FX log returns   R-squared:                       0.004
Mean Model:                                   AR   Adj. R-squared:                  0.004
Vol Model:                                EGARCH   Log-Likelihood:               -24668.1
Distribution:      Standardized Skew Student's t   AIC:                           49352.2
Method:                       Maximum Likelihood   BIC:                           49403.5
                                                   No. Observations:                 4511
Date:                           Sat, Sep 12 2020   Df Residuals:                     4503
Time:                                   22:31:29   Df Model:                            8
                                  Mean Model                                 
                    coef    std err          t      P>|t|    95.0% Conf. Int.
----------------------------------

                               AR-X - EGARCH Model Results                               
Dep. Variable:                    FX log returns   R-squared:                       0.051
Mean Model:                                 AR-X   Adj. R-squared:                  0.050
Vol Model:                                EGARCH   Log-Likelihood:               -24591.1
Distribution:      Standardized Skew Student's t   AIC:                           49206.3
Method:                       Maximum Likelihood   BIC:                           49283.2
                                                   No. Observations:                 4510
Date:                           Sat, Sep 12 2020   Df Residuals:                     4498
Time:                                   22:31:30   Df Model:                           12
                                            Mean Model                                            
                                      coef    std err          t      P>|t|       95.0% Con

## Table 1: Results of the GARCH Estimates for different specifications

In [54]:
new_index = ['Intercept', 'Lag FX log returns', 'Bid-ask spread abs value', 'Forward points first difference',
            'Interbank rate vs Libor', 'FX intervention in USD lag',
             'VIX first diff', 'EURUSD log returns', 
             'FX intervention dummy lag',
            'Omega', 'Alpha', 'Gamma', 'Beta', 'Nu', 'Lambda', 
            'R2', 'R2 adjusted', 'Number of observations', 
            'Pvalue in parenthesis', 'Significance *10%, **5%, ***1%']

dsumf = dsum.loc[new_index, :].fillna('')

# Export to latex
tex_f = os.path.join('output', 'regressions_table.tex')
dsumf.fillna('').to_latex(tex_f)

# Print
dsumf

,Constant,Microstructure,CIP,FXI,Baseline,Robustness
Intercept,1.09(0.16),-2.16(0.16),2.15(0.3),1.67***(0.0),1.63(0.45),1.64***(0.0)
Lag FX log returns,0.09***(0.0),0.08***(0.0),0.08***(0.0),0.08***(0.0),0.08***(0.0),0.08***(0.0)
Bid-ask spread abs value,,0.11**(0.01),0.15***(0.0),0.14***(0.0),0.15***(0.0),0.15***(0.0)
Forward points first difference,,0.32***(0.0),0.32***(0.0),0.32***(0.0),0.27***(0.0),0.27***(0.0)
Interbank rate vs Libor,,,-1.11***(0.0),-0.98***(0.0),-1.02***(0.01),-1.03***(0.0)
FX intervention in USD lag,,,,0.04(0.14),0.04(0.14),
VIX first diff,,,,,9.78***(0.0),9.79***(0.0)
EURUSD log returns,,,,,0.13***(0.0),0.13***(0.0)
FX intervention dummy lag,,,,,,4.13(0.22)
Omega,0.15***(0.0),0.14***(0.0),0.13***(0.01),0.13***(0.01),0.14***(0.01),0.14***(0.01)


In [53]:
# Export the frame to LateX without pvalues for beamer (shorter tables)
short_new_index = [x for x in new_index if x in dsum_short.index]
dsum_short_f = dsum_short.loc[short_new_index, :].copy()
tex_short_f = os.path.join('output', 'regressions_table_short.tex')
dsum_short_f.fillna('').to_latex(tex_short_f)

## Baseline GARCH Model: Fit and Forecast

In [ ]:
#### Specify the model
exog_l = ['Bid-ask spread abs value',
          'Forward points first difference', 
          'Interbank rate vs Libor',
          'VIX first diff',
          'EURUSD log returns',
]

dg = DistGARCH(depvar_str='FX log returns',
               data=df,
               level_str='FX level', 
               exog_l=exog_l, 
               lags_l=[1], 
               vol_model=EGARCH(1,1,1),
               # ARCH(1,1), EGARCH(1,1,1), GARCH(1,1),
               # EWMAVariance(None), RiskMetrics2006(),
               dist_family=SkewStudent(),
               # Normal, StudentsT, SkewStudent(), GeneralizedError()
)

# Fit the model
dgf = dg.fit()

# Forecast for 2020
dgfor = dgf.forecast('2020-01-01', horizon=1)


### Figure 2. Mexican Peso Against US Dollar

In [ ]:
dg.plot_description(title_level='Historical FX level',
                    title_returns='Historical FX returns',
                    title_density='Historical FX returns distribution',
                    font_scale=3)

plt.show()

### Figure 3. Conditional FX Volatility Over Time

In [ ]:
dgf.plot_in_cond_vol(start_date=None,
                     title='In-sample FX returns conditional volatility',
                     font_scale=3)
plt.show()


### Figure 4. Out-of-sample Conditional Density

In [ ]:
dgfor.joyplot_out(
    title='Out-of-sample FX returns parametric conditional density',
    xlabel='',
    label_drop=8,
    xlimits_t=(-1000, 1000),
    font_scale=4)

plt.show()


### Figure 5. Out-of-Sample Fan Chart

In [ ]:
dgfor.plot_fan_chart(font_scale=4, xticks_freq=28)
plt.show()


### Figure 6. Probability Integral Transform Test

In [ ]:
dgfor.pit_plot(font_scale=4)
plt.show()


### Figure 7. VaR FX Interventions Rule Based on a Given Information Set

In [ ]:
dgfor.plot_pdf_rule(q_low=0.05, q_high=0.975, font_scale=3)
plt.show()


### Figure 8. Conditional CDF, Out-of-sample

In [ ]:
dgfor.plot_conditional_cdf(font_scale=3, q_low=0.025, q_high=0.975, size=300)
plt.show()


### Figure 9. Conditional VaR Exceedance, Out-of-Sample

In [ ]:
dgfor.plot_var_exceedance(qv_l=[0.025, 0.975], 
                          title_1= ('Log returns and conditional VaR exceedance at 5%'
                        ' \n (green dot: <= VaR 2.5%, red dot: >= VaR 97.5%)'),
                          title_2='Corresponding FX level',
                          swap_color=True, 
                          font_scale=4, size=300)
plt.show()



## Benchmarking exercise

In [ ]:
# Read the interventions dataframe
dmx = pd.read_csv(os.path.join('data', 'intervention_data.csv'), parse_dates=['date'], index_col=0)

# Merge with the MXN peso database
dfxi = pd.merge(df, dmx[['Sell amount', 'type']],
                left_index=True, right_index=True, how='outer')

# Forecast over the Banco Mexico intervention period for benchmarking
dgmfor = dgf.forecast(dmx.index.min() - relativedelta(days=1), horizon=1)

# Include the forecasted results within the original dataframe
dfxi = dfxi.merge(dgmfor.dfor, left_index=True, right_index=True, how='outer')

# New variables
dfxi['FXI'] = (dfxi['Sell amount'] >0)
cond_fxi = (dfxi['FXI']==True)

dfxi['FXI current rate'] = np.nan
dfxi.loc[cond_fxi, 'FXI current rate'] = dfxi.loc[cond_fxi, 'FX level'].copy()

dfxi['FXI current logret'] = np.nan
dfxi.loc[cond_fxi,'FXI current logret'] = dfxi.loc[cond_fxi, 'FX log returns']

dfxi['FXI cdf'] = np.nan
dfxi.loc[cond_fxi, 'FXI cdf'] = dfxi.loc[cond_fxi, 'norm_true_val_cdf']

# Differentiate between intervention types
dmfxi = dfxi.loc[dmx.index.min():dmx.index.max(), :].copy()
dfr = dmfxi.loc[dmfxi["type"]=='min price', :].copy() # Rule
dfd = dmfxi.loc[dmfxi["type"]=='no min price', :].copy() # Discretion


### Figure 10. Rule-based FX Interventions on the Mexican Peso/USD

In [ ]:
sns.set(style='white', font_scale=4, palette='deep', font='Arial')

# With minimum price (== rule based)
fig, (ax1, ax2) = plt.subplots(2,1)

# Log returns
ax1.plot(dmfxi.index, dmfxi['FX log returns'], color='#004C97')
ax1.scatter(dfr.index, dfr['FXI current logret'], s=100, color='red', marker='D')

ax1.set_title('FX interventions and FX log returns with minimum price '
              '\n (sell USD)',
              y=1.02)
ax1.set_xlabel('')
ax1.set_ylabel('Log returns', labelpad=10)

# Level
ax2.plot(dmfxi.index, dmfxi['FX level'], color='#004C97')
ax2.scatter(dfr.index, dfr['FXI current rate'], s=100, color='red', marker='D')

ax2.set_title('FX interventions and FX level (sell USD)',
              y=1.02)
ax2.set_xlabel('')
ax2.set_ylabel('FX level', labelpad=10)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
plt.show()


### Figure 11. Conditional CDF and Rule-based FX Interventions

In [ ]:
fig, ax1 = plt.subplots(1,1)

# Local merge to have the same axis
dr = pd.merge(dmfxi['norm_true_val_cdf'], dfr['FXI cdf'], left_index=True, right_index=True, how='left')

# Level plot
ax1.plot(dr.index, 100*dr['norm_true_val_cdf'], lw=1)
ax1.scatter(dr.index, 100*dr['FXI cdf'], alpha=1, c='red', marker='D', s=300)
ax1.set_title('Conditional CDF of FX interventions with minimum price \n (sell USD)',
              y=1.02)
ax1.set_xlabel('')
ax1.set_ylabel('Percentiles', labelpad=10)

ax1.axhline(y=5, color='red', linestyle='--', lw=2)
ax1.axhline(y=95, color='red', linestyle='--', lw=2)
ax1.axhline(y=50, color='black', linestyle='-', lw=1)

# Add the ticks, if needed
ax1.set_yticks([]) # Remove the standard y ticks
new_t_l = [5, 95]
new_ticks_l = sorted(list(ax1.get_yticks()) + new_t_l)
extra_idx_l = [new_ticks_l.index(x) for x in new_t_l]
ax1.set_yticks(new_ticks_l) # Add new ticks

ax1.set_ylim(-5, 105)
plt.show()




### Figure 12. Discretionary FX Interventions on the Mexican Peso/USD

In [ ]:
sns.set(style='white', font_scale=4, palette='deep', font='Arial')
fig, (ax1, ax2) = plt.subplots(2,1)

# Log returns
ax1.plot(dmfxi.index, dmfxi['FX log returns'], color='#004C97')
ax1.scatter(dfd.index, dfd['FXI current logret'],
            s=100, color='green',
            marker='D')

ax1.set_title('FX interventions and FX log returns with no minimum price '
              '\n (sell USD)',
              y=1.02)
ax1.set_xlabel('')
ax1.set_ylabel('Log returns', labelpad=10)

# Level
ax2.plot(dmfxi.index, dmfxi['FX level'], color='#004C97')
ax2.scatter(dfd.index, dfd['FXI current rate'], s=100, color='green',
            marker='D')

ax2.set_title('FX interventions and FX level (sell USD)',
              y=1.02)
ax2.set_xlabel('')
ax2.set_ylabel('FX level', labelpad=10)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
plt.show()


### Figure 13. Conditional CDF and Discretionary FX Interventions

In [ ]:
# Local merge to have the same axis
dd = pd.merge(dmfxi['norm_true_val_cdf'], dfd['FXI cdf'],
              left_index=True, right_index=True, how='left')

fig, ax1 = plt.subplots(1,1)
# Level plot
ax1.plot(dd.index, 100*dd['norm_true_val_cdf'], lw=1)
ax1.scatter(dd.index, 100*dd['FXI cdf'], alpha=0.8, c='green', marker='o', s=200)
ax1.set_title('Conditional CDF of FX interventions with no minimum price \n (sell USD)',
              y=1.02)
ax1.set_xlabel('')
ax1.set_ylabel('Percentiles', labelpad=10)

ax1.axhline(y=5, color='red', linestyle='--', lw=2)
ax1.axhline(y=95, color='red', linestyle='--', lw=2)
ax1.axhline(y=50, color='black', linestyle='-', lw=1)

# Add the ticks, if needed
ax1.set_yticks([]) # Remove the standard y ticks
new_t_l = [5, 95]
new_ticks_l = sorted(list(ax1.get_yticks()) + new_t_l)
extra_idx_l = [new_ticks_l.index(x) for x in new_t_l]
ax1.set_yticks(new_ticks_l) # Add new ticks

ax1.set_ylim(-5, 105)

plt.show()


#### The file ends here